In [1]:
import pandas as pd
from arcgis import GIS
gis = GIS()
import requests
import arcpy
from arcgis.features import FeatureLayer
import re

# Make Feature Layer

In [2]:
sdfCombinedForecasts = pd.DataFrame.spatial.from_featureclass('results/Traffic-Volume-Historic-and-Forecast.shp')

In [3]:
sdfCombinedForecasts = sdfCombinedForecasts[sdfCombinedForecasts['ROUTE']!='UTA']
sdfCombinedForecasts

,FID,Id,SEGID,ROUTE,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,...,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_MAXMO,FAC_MAX,FACMANADJ,SHAPE
0,0,0,0039_010.7,0039,10.679,13.792,3.115424,57.0,0.0,UDOT,...,1.2056,1.1437,1.0572,1.0385,0.9133,0.8849,7.0,1.2056,0.0,"{""paths"": [[[429484.3499999996, 4567268.140000..."
1,1,0,0039_013.8,0039,13.792,16.561,2.774178,57.0,0.0,UDOT,...,1.2056,1.1437,1.0572,1.0385,0.9133,0.8849,7.0,1.2056,0.0,"{""paths"": [[[433479.0800000001, 4566770.890000..."
2,2,0,0039_016.6,0039,16.561,17.492,0.932776,57.0,0.0,UDOT,...,1.2056,1.1437,1.0572,1.0385,0.9133,0.8849,7.0,1.2056,0.0,"{""paths"": [[[434947.6900000004, 4566576.4], [4..."
3,3,0,0039_017.5,0039,17.492,19.324,1.834455,57.0,0.0,UDOT,...,1.2056,1.1437,1.0572,1.0385,0.9133,0.8849,7.0,1.2056,0.0,"{""paths"": [[[436208.5, 4568166.199999999], [43..."
4,4,0,0039_019.3,0039,19.324,21.439,2.123916,57.0,0.0,UDOT,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5.0,1.1065,0.0,"{""paths"": [[[439617.8099999996, 4567982.91], [..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6784,6784,0,MAG_6849,MAG,0.000,0.000,0.299553,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5.0,1.1065,0.0,"{""paths"": [[[410444.8064000001, 4459755.5428],..."
6785,6785,0,MAG_6850,MAG,0.000,0.000,0.497323,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5.0,1.1065,0.0,"{""paths"": [[[415088.00370000023, 4456424.71979..."
6786,6786,0,MAG_6851,MAG,0.000,0.000,0.516602,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5.0,1.1065,0.0,"{""paths"": [[[415170.73029999994, 4457251.986],..."
6787,6787,0,MAG_6852,MAG,0.000,0.000,2.024503,49.0,1.0,MAG,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9.0,1.0434,0.0,"{""paths"": [[[421074.06099999975, 4475350.47089..."


In [4]:
#already done in previous notebook
#sdfCombinedForecasts.spatial.to_featureclass('results/Traffic-Volume-Historic-and-Forecast.shp',sanitize_columns=False)

# Create Route Json

In [5]:
# Route Json
# R: Route Label (string, first 4 of SEGID)
# C: CO_FIPS (integer)
# label: Prefix of 
#        "I-"  + Route Label with 0s removed : for interstates
#        "Hwy" + Route Label with 0s removed : us and state routes
#        "FA"  + Route Label with 0s removed : federal aid routes
#        "WFRC/MAG/UDOT/UTA" 


dfRoutePrefix = pd.DataFrame([
    [   1,   14, 'Hwy'],
    [  15,   15, 'I-' ],
    [  16,   69, 'Hwy'],
    [  70,   70, 'I-' ],
    [  71,   79, 'Hwy'],
    [  80,   80, 'I-' ],
    [  81,   83, 'Hwy'],
    [  84,   84, 'I-' ],
    [  85,  214, 'Hwy'],
    [ 215,  215, 'I-' ],
    [ 216,  999, 'Hwy'],
    [1000, 5000, 'FA' ]
], columns=('routeFrom','routeTo','prefix'))
dfRoutePrefix

,routeFrom,routeTo,prefix
0,1,14,Hwy
1,15,15,I-
2,16,69,Hwy
3,70,70,I-
4,71,79,Hwy
5,80,80,I-
6,81,83,Hwy
7,84,84,I-
8,85,214,Hwy
9,215,215,I-


In [6]:
dfRoutes = sdfCombinedForecasts.groupby(['ROUTE','CO_FIPS'],as_index=False).agg(numSeg=('SEGID','count'))
dfRoutes['CO_FIPS'  ] = dfRoutes['CO_FIPS'].astype(int)
dfRoutes['ROUTE_numeric'] = pd.to_numeric(dfRoutes['ROUTE'], errors='coerce')

dfRoutes

,ROUTE,CO_FIPS,numSeg,ROUTE_numeric
0,0006,49,36,6.0
1,0007,53,14,7.0
2,0008,53,3,8.0
3,0009,53,30,9.0
4,0013,3,34,13.0
...,...,...,...,...
1029,UDOT,53,2,NaN
1030,WFRC,3,24,NaN
1031,WFRC,11,47,NaN
1032,WFRC,35,300,NaN


In [7]:
# Step 1: Add a dummy key to both DataFrames
dfRoutes['key'] = 1
dfRoutePrefix['key'] = 1

# Step 2: Merge on the dummy key
merged_df = pd.merge(dfRoutes, dfRoutePrefix, on='key')

# Step 3: Filter rows where ROUTE_numeric falls within the routeFrom and routeTo range
# Modify the filter to preserve NaN values (Step 3)
# We use | (OR) to add a condition that explicitly checks for NaNs in ROUTE_numeric
dfRoutePrefixWithPrefix = merged_df[((merged_df['ROUTE_numeric'] >= merged_df['routeFrom']) & 
                                     (merged_df['ROUTE_numeric'] <= merged_df['routeTo'])) |
                                     (merged_df['ROUTE_numeric'].isna())].copy()

# Fill specific columns with '' where ROUTE_numeric is NaN
# Adjust 'your_column_name_here' to the actual column(s) you want to fill for NaN ROUTE_numeric rows
dfRoutePrefixWithPrefix.loc[dfRoutePrefixWithPrefix['ROUTE_numeric'].isna(), 'prefix'   ] = dfRoutePrefixWithPrefix['ROUTE']
dfRoutePrefixWithPrefix.loc[dfRoutePrefixWithPrefix['ROUTE_numeric'].isna(), 'routeFrom'] = 0
dfRoutePrefixWithPrefix.loc[dfRoutePrefixWithPrefix['ROUTE_numeric'].isna(), 'routeTo'  ] = 0

dfRoutePrefixWithPrefix['ROUTE_numeric'].fillna(0, inplace=True)

dfRoutePrefixWithPrefix = dfRoutePrefixWithPrefix.drop_duplicates()

# Step 4: Drop the dummy key column
dfRoutePrefixWithPrefix = dfRoutePrefixWithPrefix.drop(columns=['key'])
dfRoutePrefixWithPrefix

,ROUTE,CO_FIPS,numSeg,ROUTE_numeric,routeFrom,routeTo,prefix
0,0006,49,36,6.0,1,14,Hwy
12,0007,53,14,7.0,1,14,Hwy
24,0008,53,3,8.0,1,14,Hwy
36,0009,53,30,9.0,1,14,Hwy
48,0013,3,34,13.0,1,14,Hwy
...,...,...,...,...,...,...,...
12348,UDOT,53,2,0.0,0,0,UDOT
12360,WFRC,3,24,0.0,0,0,WFRC
12372,WFRC,11,47,0.0,0,0,WFRC
12384,WFRC,35,300,0.0,0,0,WFRC


In [8]:
dfRoutePrefixWithPrefix['ROUTE_text'] = dfRoutePrefixWithPrefix['ROUTE_numeric'].apply(lambda x: str(int(x)) if pd.notnull(x) else np.nan)
dfRoutePrefixWithPrefix.loc[(dfRoutePrefixWithPrefix['ROUTE_text']=="0"), 'ROUTE_text'] = ''
dfRoutePrefixWithPrefix

,ROUTE,CO_FIPS,numSeg,ROUTE_numeric,routeFrom,routeTo,prefix,ROUTE_text
0,0006,49,36,6.0,1,14,Hwy,6
12,0007,53,14,7.0,1,14,Hwy,7
24,0008,53,3,8.0,1,14,Hwy,8
36,0009,53,30,9.0,1,14,Hwy,9
48,0013,3,34,13.0,1,14,Hwy,13
...,...,...,...,...,...,...,...,...
12348,UDOT,53,2,0.0,0,0,UDOT,
12360,WFRC,3,24,0.0,0,0,WFRC,
12372,WFRC,11,47,0.0,0,0,WFRC,
12384,WFRC,35,300,0.0,0,0,WFRC,


In [9]:
dfRoutePrefixWithPrefix.fillna('',inplace=True)
dfRoutePrefixWithPrefix['label']=dfRoutePrefixWithPrefix['prefix'] + dfRoutePrefixWithPrefix['ROUTE_text']

dfRoutePrefixWithPrefix.rename(columns={'ROUTE':'R','CO_FIPS':'C'}, inplace=True)

dfRoutePrefixWithPrefix = dfRoutePrefixWithPrefix[['R','C','label']]

dfRoutePrefixWithPrefix

,R,C,label
0,0006,49,Hwy6
12,0007,53,Hwy7
24,0008,53,Hwy8
36,0009,53,Hwy9
48,0013,3,Hwy13
...,...,...,...
12348,UDOT,53,UDOT
12360,WFRC,3,WFRC
12372,WFRC,11,WFRC
12384,WFRC,35,WFRC


In [10]:
# export to JSON
dfRoutePrefixWithPrefix.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/routes.json', orient='records', indent=4)

# Create Forecasts json

In [11]:
# Assuming sdfCombinedForecasts is your DataFrame

# Regular expression to match "F" followed by 4 digits, total length 5 characters
pattern = r'^F\d{4}$'

# Get a list of all columns that match the pattern (5 characters long: "F" + 4 digits)
columns_matching_pattern = [col for col in sdfCombinedForecasts.columns if re.match(pattern, col) and len(col) == 5]

# Display the list of columns
print(columns_matching_pattern)

# If you want to create a new DataFrame with just these columns
sdfCombinedForecasts_justneededcols = sdfCombinedForecasts[['SEGID','CO_FIPS'] + columns_matching_pattern]

# Now df_filtered_with_pattern contains only the columns from sdfCombinedForecasts that are 5 characters long, start with "F", and are followed by 4 digits
sdfCombinedForecasts_justneededcols

['F2019', 'F2023', 'F2028', 'F2032', 'F2042', 'F2050']


,SEGID,CO_FIPS,F2019,F2023,F2028,F2032,F2042,F2050
0,0039_010.7,57.0,9056.0,9200.0,9500.0,9700.0,10000.0,10500.0
1,0039_013.8,57.0,4185.0,4500.0,4900.0,5400.0,6400.0,7100.0
2,0039_016.6,57.0,3950.0,4200.0,4500.0,4700.0,5700.0,6300.0
3,0039_017.5,57.0,6903.0,7600.0,8400.0,8700.0,10500.0,12000.0
4,0039_019.3,57.0,2438.0,2500.0,2700.0,2800.0,3200.0,3500.0
...,...,...,...,...,...,...,...,...
6784,MAG_6849,49.0,0.0,0.0,0.0,0.0,0.0,5800.0
6785,MAG_6850,49.0,0.0,0.0,0.0,0.0,0.0,1200.0
6786,MAG_6851,49.0,0.0,0.0,0.0,350.0,1000.0,1000.0
6787,MAG_6852,49.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Assuming sdfCombinedForecasts_justneededcols is your DataFrame and columns_matching_pattern contains the columns to melt

# Melting the DataFrame
melted_df = pd.melt(sdfCombinedForecasts_justneededcols, id_vars=[col for col in sdfCombinedForecasts_justneededcols.columns if col not in columns_matching_pattern], 
                    value_vars=columns_matching_pattern, var_name='Year', value_name='Value')

# Extracting the year from the 'Year' column (assuming every value_var column is like 'F2021', 'F2022', etc.)
melted_df['Year'] = melted_df['Year'].apply(lambda x: x[1:]).astype(int)

melted_df['F'] = melted_df['Value'].astype(int)

melted_df['CO_FIPS'] = melted_df['CO_FIPS'].astype(int)

melted_df.drop(columns='Value', inplace=True)

melted_df.rename(columns={'SEGID':'S','Year':'Y'}, inplace=True)

# Now, melted_df contains the melted data with 'Year' as one of the columns
sdfCombinedForecasts_justneededcols_melted = melted_df[['S','Y','F','CO_FIPS']]
sdfCombinedForecasts_justneededcols_melted

,S,Y,F,CO_FIPS
0,0039_010.7,2019,9056,57
1,0039_013.8,2019,4185,57
2,0039_016.6,2019,3950,57
3,0039_017.5,2019,6903,57
4,0039_019.3,2019,2438,57
...,...,...,...,...
40399,MAG_6849,2050,5800,49
40400,MAG_6850,2050,1200,49
40401,MAG_6851,2050,1000,49
40402,MAG_6852,2050,0,49


In [13]:
# export to JSON
sdfCombinedForecasts_justneededcols_melted.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/forecasts.json', orient='records', indent=4)

# Create Observed

In [14]:
sdfCombinedForecasts.columns

Index(['FID', 'Id', 'SEGID', 'ROUTE', 'BMP', 'EMP', 'DISTANCE', 'CO_FIPS',
       'SUBAREAID', 'PLANAREA',
       ...
       'FAC_JUL', 'FAC_AUG', 'FAC_SEP', 'FAC_OCT', 'FAC_NOV', 'FAC_DEC',
       'FAC_MAXMO', 'FAC_MAX', 'FACMANADJ', 'SHAPE'],
      dtype='object', length=122)

In [15]:
# Assuming sdfCombinedForecasts is your DataFrame

# Regular expression to match "F" followed by 4 digits, total length 5 characters
pattern = r'^AADT\d{4}$'

# Get a list of all columns that match the pattern (5 characters long: "F" + 4 digits)
columns_matching_pattern_AADT = [col for col in sdfCombinedForecasts.columns if re.match(pattern, col) and len(col) == 8]

# Display the list of columns
print(columns_matching_pattern_AADT)

# If you want to create a new DataFrame with just these columns
sdfCombinedForecasts_justAADTneededcols = sdfCombinedForecasts[['SEGID'] + columns_matching_pattern_AADT]

# Now df_filtered_with_pattern contains only the columns from sdfCombinedForecasts that are 5 characters long, start with "F", and are followed by 4 digits
sdfCombinedForecasts_justAADTneededcols

['AADT2022', 'AADT2021', 'AADT2020', 'AADT2019', 'AADT2018', 'AADT2017', 'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013', 'AADT2012', 'AADT2011', 'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007', 'AADT2006', 'AADT2005', 'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001', 'AADT2000', 'AADT1999', 'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995', 'AADT1994', 'AADT1993', 'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989', 'AADT1988', 'AADT1987', 'AADT1986', 'AADT1985', 'AADT1984', 'AADT1983', 'AADT1982', 'AADT1981']


,SEGID,AADT2022,AADT2021,AADT2020,AADT2019,AADT2018,AADT2017,AADT2016,AADT2015,AADT2014,...,AADT1990,AADT1989,AADT1988,AADT1987,AADT1986,AADT1985,AADT1984,AADT1983,AADT1982,AADT1981
0,0039_010.7,8878.0,8463.0,9563.0,9056.0,8936.0,8640.0,8393.0,8005.0,7605.0,...,6035.0,6505.0,6510.0,6450.0,6000.0,5780.0,5665.0,6310.0,6215.0,6195.0
1,0039_013.8,4102.0,3911.0,4419.0,4185.0,4131.0,3995.0,3882.0,3700.0,3515.0,...,2870.0,3085.0,0.0,6450.0,6000.0,5780.0,5665.0,6310.0,6215.0,6195.0
2,0039_016.6,3872.0,3691.0,4171.0,3950.0,3899.0,3771.0,3620.0,3450.0,3275.0,...,1850.0,1990.0,1665.0,1660.0,1630.0,1600.0,1565.0,2165.0,2135.0,2115.0
3,0039_017.5,6768.0,6452.0,7290.0,6903.0,6814.0,6590.0,6404.0,6105.0,5800.0,...,1850.0,1990.0,1665.0,1660.0,1630.0,1600.0,1565.0,2165.0,2135.0,2115.0
4,0039_019.3,2765.0,2754.0,2618.0,2438.0,2465.0,2375.0,2297.0,2220.0,2065.0,...,880.0,895.0,880.0,845.0,830.0,815.0,800.0,900.0,870.0,860.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6784,MAG_6849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6785,MAG_6850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6786,MAG_6851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6787,MAG_6852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
sdfCombinedForecasts_justAADTneededcols.columns

Index(['SEGID', 'AADT2022', 'AADT2021', 'AADT2020', 'AADT2019', 'AADT2018',
       'AADT2017', 'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013', 'AADT2012',
       'AADT2011', 'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007', 'AADT2006',
       'AADT2005', 'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001', 'AADT2000',
       'AADT1999', 'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995', 'AADT1994',
       'AADT1993', 'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989', 'AADT1988',
       'AADT1987', 'AADT1986', 'AADT1985', 'AADT1984', 'AADT1983', 'AADT1982',
       'AADT1981'],
      dtype='object')

In [17]:
# Assuming sdfCombinedForecasts_justneededcols is your DataFrame and columns_matching_pattern contains the columns to melt

# Melting the DataFrame
melted_df = pd.melt(sdfCombinedForecasts_justAADTneededcols, id_vars=[col for col in sdfCombinedForecasts_justAADTneededcols.columns if col not in columns_matching_pattern_AADT], 
                    value_vars=columns_matching_pattern_AADT, var_name='Year', value_name='Value')

# Extracting the year from the 'Year' column (assuming every value_var column is like 'F2021', 'F2022', etc.)
melted_df['Year'] = melted_df['Year'].apply(lambda x: x[4:]).astype(int)

melted_df = melted_df[melted_df['Year']>=2000]

melted_df['O'] = melted_df['Value'].astype(int)

melted_df.drop(columns='Value', inplace=True)

melted_df.rename(columns={'SEGID':'S','Year':'Y'}, inplace=True)

# Now, melted_df contains the melted data with 'Year' as one of the columns
sdfCombinedForecasts_justAADTneededcols_melted = melted_df[['S','Y','O']]
sdfCombinedForecasts_justAADTneededcols_melted

,S,Y,O
0,0039_010.7,2022,8878
1,0039_013.8,2022,4102
2,0039_016.6,2022,3872
3,0039_017.5,2022,6768
4,0039_019.3,2022,2765
...,...,...,...
154877,MAG_6849,2000,0
154878,MAG_6850,2000,0
154879,MAG_6851,2000,0
154880,MAG_6852,2000,0


In [18]:
sdfCombinedForecasts_justAADTneededcols_melted[sdfCombinedForecasts_justAADTneededcols_melted['S']=='0015_000.0']

,S,Y,O
237,0015_000.0,2022,30108
6971,0015_000.0,2021,29259
13705,0015_000.0,2020,23546
20439,0015_000.0,2019,24681
27173,0015_000.0,2018,24316
33907,0015_000.0,2017,23224
40641,0015_000.0,2016,21992
47375,0015_000.0,2015,20195
54109,0015_000.0,2014,18910
60843,0015_000.0,2013,18630


In [19]:
# export to JSON
sdfCombinedForecasts_justAADTneededcols_melted.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/observed.json', orient='records', indent=0)

# create segments json

In [20]:
dfSegments = pd.merge(sdfCombinedForecasts[['SEGID','ROUTE','BMP']],dfRoutePrefixWithPrefix, left_on='ROUTE', right_on='R')
dfSegments.rename(columns={'SEGID':'S','label':'N'},inplace=True)

# set label
dfSegments['L'] = dfSegments['N'] + ' ~MP' + dfSegments['BMP'].round(1).astype(str)
dfSegments.loc[(dfSegments['R'].isin(['WFRC','MAG','CACHE','DIXIE','UDOT','UTA'])), 'L'] = dfSegments['S']


dfSegments.drop(columns=['ROUTE','BMP'],inplace=True)
dfSegments

,S,R,C,N,L
0,0039_010.7,0039,57,Hwy39,Hwy39 ~MP10.7
1,0039_013.8,0039,57,Hwy39,Hwy39 ~MP13.8
2,0039_016.6,0039,57,Hwy39,Hwy39 ~MP16.6
3,0039_017.5,0039,57,Hwy39,Hwy39 ~MP17.5
4,0039_019.3,0039,57,Hwy39,Hwy39 ~MP19.3
...,...,...,...,...,...
11049,MAG_6851,MAG,49,MAG,MAG_6851
11050,MAG_6852,MAG,23,MAG,MAG_6852
11051,MAG_6852,MAG,49,MAG,MAG_6852
11052,MAG_6853,MAG,23,MAG,MAG_6853


In [21]:
# export to JSON
dfSegments.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/segments.json', orient='records', indent=0)

In [22]:
dfSegments[dfSegments['C']==21]

,S,R,C,N,L
92,0015_368.1,0015,21,I-15,I-15 ~MP368.1
99,0015_372.7,0015,21,I-15,I-15 ~MP372.7
106,0015_376.5,0015,21,I-15,I-15 ~MP376.5
113,0015_379.3,0015,21,I-15,I-15 ~MP379.4
120,0015_380.0,0015,21,I-15,I-15 ~MP380.0
...,...,...,...,...,...
3703,UDOT_7000,UDOT,21,UDOT,UDOT_7000
3705,UDOT_7001,UDOT,21,UDOT,UDOT_7001
3707,UDOT_7006,UDOT,21,UDOT,UDOT_7006
3709,UDOT_7007,UDOT,21,UDOT,UDOT_7007
